In [2]:
import numpy as np
import os
import random
import tensorflow as tf
import cv2


In [3]:
model = tf.keras.models.load_model(os.path.join("files", "best_model.h5"))

In [4]:
IMG_SIZE = (128,128)

In [7]:
def process_image(image):
    # Preprocess the image (resize, normalize, etc.)
    image = cv2.resize(image, IMG_SIZE)
    image = image / 255.0  # Normalize pixel values if needed
    
    # Get the binary mask
    pred_mask = model.predict(np.expand_dims(image, axis=0))
    mask = np.round(pred_mask[0])
    
    # Convert the mask to an image
    zero_image = np.zeros_like(mask)
    mask = np.dstack((mask, zero_image, zero_image)) * 255
    mask = np.asarray(mask, np.uint8)
    
    # Overlay the mask on the original image
    final_image = cv2.addWeighted(image.astype(np.float32), 1.0, mask.astype(np.float32), 0.5, 0.0, dtype=cv2.CV_32F)
    
    final_image = cv2.resize(final_image, (1280, 720))
    
    return final_image

In [6]:
os.mkdir("videos")

In [8]:
input_video_path = 'C:\\Users\\adity\\Downloads\\challenge.mp4'
cap = cv2.VideoCapture(input_video_path)

output_video_path = 'videos\\output_video.mp4'

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fps = 60

out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))


# Process Video
while(cap.isOpened()):
    ret, frame = cap.read()

    if not ret:
        break  # Break the loop if no more frames

    processed_frame = process_image(frame)
    # Write the frame into the file 'output.avi'
    out.write(processed_frame)

    # Display the processed frame (optional)
    cv2.imshow('Processed Frame', processed_frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    

# When everything done, release the video capture and video write objects
cap.release()
out.release()




1/1 [==============================] - 0s 121ms/step
